This is a dummy GPT model to give a birds eye view of how the model should look like

In [1]:
import torch
import torch.nn as nn

In [2]:
GPT_CONFIG_124M = {
    "vocab_size": 50257, #vocabulary size
    "context_length": 1024, #context length
    "emb_dim": 768, #embedding dimmension
    "n_heads": 12, #number of attention heads
    "n_layers": 12, #number of tranformer layers
    "drop_rate": 0.1, #Dropout rate
    "qkv_bias": False #Query-Key-Value bias
}

GPT architecture: Dummy GPT model Class

In [3]:
#here we will be using a placehoder for both the transformer block and layer normalization

class DummyGPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"]) #converts each input to its corresponding token embedding with dimension of vocab size x vector embedding dimension (50257 x 76)
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"]) #get the position embedding, ps: postional embedding really depends on the context size
        self.drop_emb = nn.Dropout(cfg["drop_rate"]) #dropout rate

        #use a placeholder for TransformerBlock
        self.trf_blocks = nn.Sequential(
            *[DummyTransformerBlock(cfg) for _ in range(cfg["n_layers"])])

        #use a placeholder  for LayerNorm
        self.final_norm = DummyLayerNorm(cfg["emd_dim"])
        self.out_head = nn.Linear(
            cfg["emb_dim"], cfg["vocab_size"], bias=False
        )
        
    def forward(self, in_idx): #this takes in an input
        batch_size, seq_len = in_idx.shape #seq_len is the no of tokens
        tok_embeds = self.tok_emb(in_idx) #takes the input in_idx
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        logits = self.out_head(x)
        return logits
    
class DummyTransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        #a simple placeholder

    def forward(self, x):
        #this block does nothing but return the input
        return x

class DummyLayerNorm(nn.Module): #this block appears severally in the tranformer block and the GPT architectur so it is essential to define it separately. 
    def __init__(self, normalized_shape, eps=1e-5):
        super().__init__()
        #the paramters here are just to mimick the LayerNorm interface

    def forward(self, x):
        #this layer does nothing but returns its input
        return x

In [22]:
# Implementing the GELU activation function
class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(
            torch.sqrt(torch.tensor(2.0 / torch.pi, dtype=x.dtype, device=x.device)) *
            (x + 0.044715 * torch.pow(x, 3))
        ))

In [23]:
# Implementing the FeedForward Module of the transformer architecture
class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]),  # Expansion
            GELU(),  # Activation
            nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]),  # Contraction
        )

    def forward(self, x):
        return self.layers(x)

In [24]:
print(GPT_CONFIG_124M["emb_dim"])

768


The feed forward module is a simple neural network with two linear layers and one activation function.

In [26]:
ffn = FeedForward(GPT_CONFIG_124M)
x = torch.rand(2, 3, 768)
out = ffn(x)
print(out.shape)

torch.Size([2, 3, 768])
